# Streaming

Un <b>Stream</b> es un flujo de datos que no tiene fin. Como los paquetes en una red, las publicaciones en redes sociales como Twitter, Facebook, las ventas en sitios como Amazon. Son flujos de datos continuos. Para realizar consultas o calcular estadisticas, metricas a estos flujos se necesitan algoritmos especiales.

## Reservoir Sampling

Es un algoritmo que permite convertir un <b>stream</b>, que es un conjunto de datos infinitos, en un conjunto de datos finitos, que se denomina <b>sample</b> o <b>muestra</b>. Y asi luego poder aplicar las metricas o las consultas a esta muestra en lugar de hacerlo al <b>stream</b> original.

- Llamamos <b>N</b> a la cantidad de datos del Stream hasta el momento.
- El algoritmo mantiente una cantidad <b>k</b> de datos en el sample.

Se tiene que cumplir que la probabilidad de que un dato del stream ($X_i$) este en el stream sea:

\begin{equation}
P(X_i \in sample) = \frac{k}{N}
\end{equation}

Para lograr que la probabilidad sea esa el algoritmo hace lo siguiente:
Por cada dato que genera nuestro Stream, el algoritmo genera un numero aleatorio entre 0 y 1. Si el numero generado es menor a $p=\frac{k}{N}$ entonces el algoritmo ingresa a la muestra. Si tenia menos de <b>k</b> elementos se agrega sin reemplazar a ninguno, en cambio si tenia <b>k</b> elementos, reemplaza aleatoriamente a cualquiera. Si el numero generado es mayor a p entonces no entra a la muestra.

## Momentos de un Stream

$M_i$ es la cantidad de veces que el elemento i ocurrio en el stream.

<b>Momento de orden k de un Stream</b>:

\begin{equation}
M^k(S) = \sum_{i \in S} M_i^k
\end{equation}

<b>$M^0(S)$</b>: <b>Momento de orden 0</b>: Es la cantidad de elementos diferentes en un stream. <br>
<b>$M^1(S)$</b>: <b>Momento de orden 1</b>: Es la cantidad de elementos que hay en un stream. <br>
<b>$M^2(S)$</b>: <b>Momento de orden 2</b>: Tambien llamado <b>Numero Sopresa</b>. Es un indicador de si los datos se distribuyen de forma pareja.

## Flajolet-Martin

Es un algoritmo para estimar el <b>Momento de orden 0</b>, es decir la cantidad de elementos diferentes en un stream. Para hacerlo lo que hace es a cada dato que observamos en el stream le aplica una funcion de hashing. Que transforma el dato en un numero binario. Luego cuenta la cantidad de bits en 0 consecutivos a partir del bit cero, y lo guarda en un contador(<b>r</b>) si es que es el maximo.

El <b>momento de orden 0</b> del stream se estima como:

\begin{equation}
M^0(S) \approx 2^r
\end{equation}

## AMS

Es un algoritmo para estimar el <b>Momento de orden 2</b> de un stream. Mantiene en memoria <b>k</b> estimadores, cada estimador registra un elemento y la cantidad de veces que ocurre. Por cada elemento observado del stream si un estimador lo tiene registrado aumenta el contador en uno, y si ninguno lo tiene entonces ingresa a memoria con probabilidad $\frac{k}{N}$ de la misma forma que con <b>Reservoir Sampling</b>. 

Luego cada estimador estima el momento asi:

\begin{equation}
M^2(S) \approx N (2 * C_i - 1)
\end{equation}

$C_i$: Es valor del contador.

Y el <b>resultado final</b> es el promedio de los estimadores.

## DGIM

Es un algoritmo para estimar la cantidad de 1s de un <b>stream binario</b> en los ultimos <b>k</b> bits. Tomando una ventana muy grande de tamanio <b>m</b>. La idea es formar <b>sub-ventanas</b> que tengan dos datos, la posicion en donde arrancan y la cantidad de bits en 1. Se ordenan por antiguedad de mas viejas a mas nuevas. Entonces es como que se reemplaza el stream por un conjunto de ventanas. Luego nos podemos podemos preguntar cuantos 1s hay en las ultimas <b>k</b> posiciones.

Para construir las ventas se va procesando el stream, cuando aparece un 1, se crea una ventana con ese elemento. Y para cada nuevo 1 se analiza lo siguiente:

- Si la cantidad de <b>sub-ventanas</b> de un 1 bit es: <br>
 - 1 o 2 no hacemos nada.
 - Si hay 3 entonces combinamos las dos sub-ventanas mas viejas en una nueva de 2 bits.
- Si la cantidad de <b>sub-ventanas</b> de    2 bits es: <br>
 - 1 o 2 no hacemos nada.
 - Si hay 3 entonces combinamos las dos sub-ventanas mas viejas en una nueva de 4 bits.
- ...
- Si la cantidad de <b>sub-ventanas</b> de n bits es: <br>
 - 1 o 2 no hacemos nada.
 - Si hay 3 entonces combinamos las dos sub-ventanas mas viejas en una nueva de 2*n bits.

Luego, para dar la <b>respeusta final</b> de la cantidad 1s en las ultimas <b>k</b> posiciones, lo que hacemos es sumar los bits que contienen las ventanas que se encuentran en posiciones menores a <b>k</b> y a la ultima ventana dividimos al cantida de bits por dos.

## Filtros de Bloom

Es un algoritmo para estimar si un <b>elemento</b> (dato) pertenece a un <b>conjunto</b>. Por <b>ejemplo</b> tenemos una lista de mails que sabemos que son spam, entonces dado un email queremos saber si es spam, osea si pertenece a esa lista. 

El <b>Filtro de Bloom</b> es un vector de <b>m</b> bits. Donde lo que el algoritmo hace es agarrar cada elemento del conjunto y aplicarle <b>k</b> funciones de hashing que lo que hacen es setear en 1 varias posiciones de este vector filtro. Luego la manera de saber si nuestro elemento pertence al conjunto es aplicandole las <b>k</b> funciones de hashing y analizando que coincida en todos los 1s.  

Este metodo no posee <b>falsos negativos</b> pero si tiene <b>falsos positivos</b> debido a las coliciones de las funciones de hashing. La probabilidad de un <b>falso positivo</b> depende de <b>k, n y m</b>. El valor optimo de <b>k</b> es:

\begin{equation}
k = \frac{m}{n} log(2)
\end{equation}

## Count-Min

Es un algoritmo que sirve para estimar la cantidad de veces que ocurrio cada elemento de un <b>stream</b>, logrando poder averiguar los elementos mas populares. 

El algoritmo a medida que va observando los elementos del stream los hashea con <b>k</b> funciones de hashing. Y aumenta el contador de cada posicion en la tabla. Hace lo mismo para todos los elementos. Luego para obtener la cantidad de veces que ocurrio algun elemento, lo que hace es hashearlo con cada funcion de hash y recuperar el contardor, y lo que hace es quedarse con el contador que tenga el numero mas pequenio. Y esta es la respuesta del algoritmo.

<b>Ejemplo</b> de la tabla.

|h |0|1|2|3|4|5|6|
|- |-|-|-|-|-|-|-|
|h1|0|4|0|2|0|0|1|
|h2|0|0|2|0|2|0|3|
|h3|0|2|0|3|2|0|0|
|h4|0|4|0|0|0|0|3|